In [2]:
import matplotlib.pylab as plt
import seaborn as sns
from onsset import *

In [3]:
def calc_summary_table(df):

    elements = []
    for year in yearsofanalysis:
        elements.append("Population{}".format(year))
        elements.append("NewConnections{}".format(year))
        elements.append("Capacity{}".format(year))
        elements.append("Investment{}".format(year))
    
    techs = ["1", "2", "3", "5", "6", "7", "99"]
    #techs = ["Grid densification", "Grid extension", "SA_PV", "MG_Diesel", "MG_PVHybrid", "MG_Wind", "MG_Hydro"]
    
    sumtechs = []
    for year in yearsofanalysis:
        sumtechs.extend(["Population{}".format(year) + t for t in techs])
        sumtechs.extend(["NewConnections{}".format(year) + t for t in techs])
        sumtechs.extend(["Capacity{}".format(year) + t for t in techs])
        sumtechs.extend(["Investment{}".format(year) + t for t in techs])
    
    summary = pd.Series(index=sumtechs, name='country')
    
    
    for year in yearsofanalysis:
        for t in techs:
            if t == '99':
                summary.loc["Population{}".format(year) + t] = df[SET_POP + "{}".format(year)].sum() - df[SET_ELEC_POP + "{}".format(year)].sum() 
            else:
                summary.loc["Population{}".format(year) + t] = df.loc[(df[SET_ELEC_FINAL_CODE + '{}'.format(year)] == int(t)), SET_POP + '{}'.format(year)].sum()
            if t == '99':
                summary.loc["NewConnections{}".format(year) + t] = 0
            else:
                summary.loc["NewConnections{}".format(year) + t] = df.loc[(onsseter.df[SET_ELEC_FINAL_CODE + '{}'.format(year)] == int(t)), SET_NEW_CONNECTIONS + '{}'.format(year)].sum()
            summary.loc["Capacity{}".format(year) + t] = df.loc[(df[SET_ELEC_FINAL_CODE + '{}'.format(year)] == int(t)) & (df[SET_ELEC_FINAL_CODE + '{}'.format(year)] < 99), SET_NEW_CAPACITY + '{}'.format(year)].sum()/1000
            summary.loc["Investment{}".format(year) + t] = df.loc[(df[SET_ELEC_FINAL_CODE + '{}'.format(year)] == int(t)) & (df[SET_ELEC_FINAL_CODE + '{}'.format(year)] < 99), SET_INVESTMENT_COST + '{}'.format(year)].sum()
        
    summary = summary.fillna(0)
         
    index = techs + ['Total']
    columns = []
    
    for year in yearsofanalysis:
        columns.append("Population{}".format(year))
        columns.append("NewConnections{}".format(year))
        columns.append("Capacity{} (MW)".format(year))
        columns.append("Investment{} (million USD)".format(year))
    columns.append("PopulationTotal".format(year))
    columns.append("NewConnectionsTotal".format(year))
    columns.append("CapacityTotal (MW)".format(year))
    columns.append("InvestmentTotal (million USD)".format(year))
        
    summary_table = pd.DataFrame(index=index, columns=columns)
    
    summary_table[columns[0]] = summary.iloc[0:7].astype(int).tolist() + [int(summary.iloc[0:7].sum())]
    summary_table[columns[1]] = summary.iloc[7:14].astype(int).tolist() + [int(summary.iloc[7:14].sum())]
    summary_table[columns[2]] = summary.iloc[14:21].tolist() + [summary.iloc[14:21].sum()]
    summary_table[columns[3]] = [round(x/1e4)/1e2 for x in summary.iloc[21:28].astype(float).tolist()] + [round(summary.iloc[21:28].sum()/1e4)/1e2]
    summary_table[columns[4]] = summary.iloc[28:35].astype(int).tolist() + [int(summary.iloc[28:35].sum())]
    summary_table[columns[5]] = summary.iloc[35:42].astype(int).tolist() + [int(summary.iloc[35:42].sum())]
    summary_table[columns[6]] = summary.iloc[42:49].tolist() + [summary.iloc[42:49].sum()]
    summary_table[columns[7]] = [round(x/1e4)/1e2 for x in summary.iloc[49:56].astype(float).tolist()] + [round(summary.iloc[49:56].sum()/1e4)/1e2]
    summary_table[columns[8]] = summary_table[columns[4]]
    summary_table[columns[9]] = summary_table[columns[1]] + summary_table[columns[5]]
    summary_table[columns[10]] = summary_table[columns[2]] + summary_table[columns[6]]
    summary_table[columns[11]] = summary_table[columns[3]] + summary_table[columns[7]]
    
    map_dict = {'1': 'Grid_dens',
                '2': 'Grid_ext',
                '3': 'SA_PV',
                '5': 'MG_PV_Hybrid',
                '6': 'MG_Wind',
                '7': 'MG_Hydro', 
                '99':'Non-electrified',
                'Total': 'Total'}
    
    summary_table.index = summary_table.index.to_series().map(map_dict)
    
    return summary_table, columns


In [ ]:
def bar_plot(summary_table, columns):
    colors = ['#4e53de', '#a6aaff', '#ffc700', '#e628a0', '#1b8f4d', '#28e66d', '#808080']
    techs_colors = dict(zip(techs, colors))
    
    summary_plot=summary_table.drop(labels='Total',axis=0)
    fig_size = [35, 35]
    font_size = 1
    plt.rcParams["figure.figsize"] = fig_size
    f, axarr = plt.subplots(2, 2)
    fig_size = [15, 15]
    font_size = 20
    plt.rcParams["figure.figsize"] = fig_size
    
    sns.barplot(x=summary_plot.index.tolist(), y=columns[8], data=summary_plot, ax=axarr[0, 0], palette=colors, hue=colors)
    axarr[0, 0].set_ylabel(columns[8], fontsize=2*font_size)
    axarr[0, 0].tick_params(axis='x', labelrotation=45)
    axarr[0, 0].tick_params(labelsize=font_size)
    sns.barplot(x=summary_plot.index.tolist(), y=columns[9], data=summary_plot, ax=axarr[0, 1], palette=colors, hue=colors)
    axarr[0, 1].set_ylabel(columns[9], fontsize=2*font_size)
    axarr[0, 1].tick_params(axis='x', labelrotation=45)
    axarr[0, 1].tick_params(labelsize=font_size)
    sns.barplot(x=summary_plot.index.tolist(), y=columns[10], data=summary_plot, ax=axarr[1, 0], palette=colors, hue=colors)
    axarr[1, 0].set_ylabel(columns[10], fontsize=2*font_size)
    axarr[1, 0].tick_params(axis='x', labelrotation=45)
    axarr[1, 0].tick_params(labelsize=font_size)
    sns.barplot(x=summary_plot.index.tolist(), y=columns[11], data=summary_plot, ax=axarr[1, 1], palette=colors, hue=colors)
    axarr[1, 1].set_ylabel(columns[11], fontsize=2*font_size)
    axarr[1, 1].tick_params(axis='x', labelrotation=45)
    axarr[1, 1].tick_params(labelsize=font_size)

In [ ]:
def map_plot(df):
    colors = ['#4e53de', '#a6aaff', '#ffc700', '#e628a0', '#1b8f4d', '#28e66d', '#808080']
    plt.figure(figsize=(9,9))
    plt.plot(df.loc[df['FinalElecCode{}'.format(end_year)]==3, SET_X_DEG], df.loc[df['FinalElecCode{}'.format(end_year)]==3, SET_Y_DEG], color='#ffc700', marker=',', linestyle='none')
    plt.plot(df.loc[df['FinalElecCode{}'.format(end_year)]==4, SET_X_DEG], df.loc[df['FinalElecCode{}'.format(end_year)]==99, SET_Y_DEG], color='#808080', marker=',', linestyle='none')
    plt.plot(df.loc[df['FinalElecCode{}'.format(end_year)]==5, SET_X_DEG], df.loc[df['FinalElecCode{}'.format(end_year)]==5, SET_Y_DEG], color='#e628a0', marker=',', linestyle='none')
    plt.plot(df.loc[df['FinalElecCode{}'.format(end_year)]==6, SET_X_DEG], df.loc[df['FinalElecCode{}'.format(end_year)]==6, SET_Y_DEG], color='#1b8f4d', marker=',', linestyle='none')
    plt.plot(df.loc[df['FinalElecCode{}'.format(end_year)]==7, SET_X_DEG], df.loc[df['FinalElecCode{}'.format(end_year)]==7, SET_Y_DEG], color='#28e66d', marker=',', linestyle='none')
    plt.plot(df.loc[df['FinalElecCode{}'.format(end_year)]==1, SET_X_DEG], df.loc[df['FinalElecCode{}'.format(end_year)]==1, SET_Y_DEG], color='#4e53de', marker=',', linestyle='none')
    plt.plot(df.loc[df['FinalElecCode{}'.format(end_year)]==2, SET_X_DEG], df.loc[df['FinalElecCode{}'.format(end_year)]==2, SET_Y_DEG], color='#a6aaff', marker=',', linestyle='none')
    if df[SET_X_DEG].max() - df[SET_X_DEG].min() > df[SET_Y_DEG].max() - df[SET_Y_DEG].min():
        plt.xlim(df[SET_X_DEG].min() - 1, df[SET_X_DEG].max() + 1)
        plt.ylim((df[SET_Y_DEG].min()+df[SET_Y_DEG].max())/2 - 0.5*abs(df[SET_X_DEG].max() - df[SET_X_DEG].min()) - 1, (df[SET_Y_DEG].min()+df[SET_Y_DEG].max())/2 + 0.5*abs(df[SET_X_DEG].max() - df[SET_X_DEG].min()) + 1)
    else:
        plt.xlim((df[SET_X_DEG].min()+df[SET_X_DEG].max())/2 - 0.5*abs(df[SET_Y_DEG].max() - df[SET_Y_DEG].min()) - 1, (df[SET_X_DEG].min()+df[SET_X_DEG].max())/2 + 0.5*abs(df[SET_Y_DEG].max() - df[SET_Y_DEG].min()) + 1)
        plt.ylim(df[SET_Y_DEG].min() -1, df[SET_Y_DEG].max() +1)
    plt.figure(figsize=(30,30))

In [ ]:
def finalize_results(self, yearsofanalysis, simplified=False):
    if simplified:
        del self.df['minTDdist']
        del self.df['MVConnectDist']
        del self.df['AnnualEmissionsTotal']
        for year in yearsofanalysis:
            del self.df['AnnualEmissions' + "{}".format(year)]
            del self.df['PVHybridEmissionFactor' + "{}".format(year)]
    else:
        del self.df['NewDist']
        del self.df['MaxIntensificationDist']
        del self.df['GridCapacityRequired']
        del self.df['MaxDist']
        del self.df['X']
        del self.df['Y']
        del self.df['minTDdist']
        del self.df['MVConnectDist']
        del self.df['AnnualEmissionsTotal']
        for year in yearsofanalysis:
            del self.df['PreSelection' + "{}".format(year)]
            del self.df['AnnualEmissions' + "{}".format(year)]
            del self.df['GridCapacityRequired' + "{}".format(year)]
            del self.df['MaxDist' + "{}".format(year)]
            del self.df['PVHybridEmissionFactor' + "{}".format(year)]

    for i in range(len(self.df.columns)):
        if self.df.iloc[:, i].dtype == 'float64':
            self.df.iloc[:, i] = pd.to_numeric(self.df.iloc[:, i], downcast='float')
        elif self.df.iloc[:, i].dtype == 'int64':
            self.df.iloc[:, i] = pd.to_numeric(self.df.iloc[:, i], downcast='signed')